# Segregación de Schelling

In [1]:
import pandas as pd
pd.set_option('plotting.backend', 'pandas_bokeh')
pd.plotting.output_notebook()

from shelling.model import City

Loading BokehJS ...

El modelo propuesto originalmente por Thomas Schelling consiste en dos grupos de agentes, por ejemplo, rojos y verdes, que localmente tratan de satisfacer la necesidad de estar con los de su mismo grupo.

De manera general este comportamiento es establecido con un parámetro conocido como porcentaje de similitud-requerida o nivel de tolerancia.

Los agentes toman una decisión a partir de la información que tienen en su vecindad. Si el agente satisface las condiciones del entorno entonces se queda en su posición actual, de lo contrario, se mueve a una posición vacía.

Esta dinámica local genera como resultado la formación de cúmulos de agentes del mismo tipo, es decir, hay segregación.

**Definición del sistema (entorno)**: El sistema se compone de una retícula de $n\times n$ donde n se establece usualmente entre 50 y 100. Cada celda con posición $(i,j)$ alberga a un agente rojo o verde. El sistema tiene un parámetro de densidad poblacional, usualmente se establece en 90% (es decir, 10% de las celdas quedan vacías). La mitad de la población se inicializa como rojos y la otra como verdes. Cada agente toma una celda de manera aleatoria.

**Dinámica (reglas)**: cada agente en la posición $(i,j)$ se “muda” a un lugar vacío si en su vecindad de Moore (8-vecinos) no cumple con el porcentaje de similitud requerida.

In [2]:
def run_and_plot(city, steps=100):
    for _ in range(steps):
        if city.running:
            city.step()
            
    df = city.datacollector.get_model_vars_dataframe()
    df.plot(
        legend=False,
        xlabel='steps',
        ylabel='satisfied citizens',
        xticks=range(0, df.size+1, 10)
    )
    
def build_city(sim):
    return City(
        density = 0.9, 
        width= 50,
        height= 50,
        sim_type='constant',
        sim_percentage = sim
    )

In [3]:
city_example= build_city(0.75)

In [4]:
run_and_plot(city_example, steps=120)

## Límite de similitud

Establezca el tamaño de retícula como $n=50$, con densidad poblacional del 90% ¿Qué valor del parámetro de similitud es el límite máximo para formar dinámicas de segregación? A este valor le llamaremos $S_{max}$.

## Convergencia

Una propuesta de medida para detectar convergencia es cuando los agentes ya no cambian de posición.
Cuándo el parámetro de similitud es igual a $S_{max}$, ¿cuál es el tiempo en el que el sistema converge?

De manera general, forme una gráfica parámetro-similitud vs tiempo-de-convergencia.

¿Cómo crece/decrece el tiempo de convergencia en función del parámetro de similitud? ¿lineal, logarítmico, exponencial?

Cuando el sistema no converja (probar un tiempo suficientemente grande) dejar de graficar.

In [5]:
from mesa.batchrunner import BatchRunner

In [6]:
import numpy as np

In [7]:
fixed_params = {
    'density': 0.9,
    'width': 50,
    'height': 50,
    'sim_type': 'constant'
}

var_params = {
    'sim_percentage': np.linspace(0, 0.75, 76)
}

runner = BatchRunner(
    City,
    var_params,
    fixed_params,
    iterations=1,
    max_steps=400,
    model_reporters={
        'convergence_time': lambda m: m.steps
    }
)

In [8]:
runner.run_all()

76it [05:47,  4.57s/it]


In [9]:
conv_time_df = runner.get_model_vars_dataframe()
conv_time_df.plot.scatter(
    legend=False,
    x='sim_percentage', y='convergence_time',
    xlabel='similitud percentage',
    ylabel='convergence steps',
    xticks=np.linspace(0, 0.75, 16)
)

Figure(id='1097', ...)

## Similitud normal

Establezca el parámetro de similitud como un atributo de los agentes. Inicialice la similitud requerida del agente $i$-ésimo a partir de una distribución normal con media 50 y desviación estándar 10.

¿Cómo cambian los patrones de segregación? 

In [10]:
def build_normal_city(m, s):
    return City(
        density = 0.9, 
        width= 50,
        height= 50,
        sim_type='normal',
        mean = m,
        deviation = s
    )

In [11]:
normal_city = build_normal_city(0.5, 0.1)

In [12]:
run_and_plot(normal_city)

In [13]:
city_const= build_city(0.5)

In [14]:
run_and_plot(city_const)

¿Y con $media = S_{max}$ y desviación estándar pequeña y grande?

In [15]:
city0_n_smax = build_normal_city(0.75, 0.0001)

In [16]:
run_and_plot(city0_n_smax, steps=150)

In [17]:
city1_n_smax = build_normal_city(0.75, 0.5)

In [18]:
run_and_plot(city1_n_smax, steps=150)

## Tres grupos

Modifique su programa previo para considerar tres tipos de agentes (rojos, verdes y azules). Inicialice cada grupo como $\frac{1}{3}$ de la población y establezca de manera global el parámetro de similitud requerida.

¿Se forman patrones de segregación? ¿Cuál es el valor del umbral $S_{max}$?

In [19]:
def build_3_city(sim):
    return City(
        density = 0.9, 
        width= 50,
        height= 50,
        num_groups=3,
        sim_type='constant',
        sim_percentage=sim
    )

In [20]:
city3 = build_3_city(0.625)

In [21]:
run_and_plot(city3, steps=150)

In [22]:
city3_bad = build_3_city(0.6251)

In [23]:
run_and_plot(city3_bad, steps=150)